In [7]:
import numpy as np
from IPython.display import clear_output

w1 = np.array([[-125],[0],[ 0 ],[1]])
w2 = np.array([[125 ],[0],[45 ],[1]])
w3 = np.array([[125 ],[0],[-45],[1]])

def trans_matrix(t):
    M = np.eye(4,4)
    M[0,3] = t[0,0]
    M[1,3] = t[1,0]
    M[2,3] = t[2,0]
    return M
# trans_matrix(t1)

def rot_matrix(angle,axis):
    if axis == 'x':
        R = np.array([
        [1, 0, 0, 0],
        [0,np.cos(angle),-np.sin(angle), 0],
        [0,np.sin(angle), np.cos(angle), 0],
        [0, 0, 0, 1]])
    elif axis == 'y':
        R = np.array([
        [np.cos(angle), 0, np.sin(angle), 0],
        [0, 1, 0, 0],
        [-np.sin(angle), 0, np.cos(angle), 0],
        [0, 0, 0, 1]])
    elif axis == 'z':
        R = np.array([
        [np.cos(angle),-np.sin(angle),0, 0],
        [np.sin(angle), np.cos(angle),0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])
    else:
        R = np.eye(4,4)
        print('invalid axis given')
    return R

In [8]:
# o : orgin on plate(0,0,0) , m : magnet center(0,205,0)
o = np.array([[0],[0],[0],[1]])
m = np.array([[0],[205],[0],[1]])

# Calculating the angle'
def change_lay3(n1,n2,n3):
    d1 = 250
    d2 = 88.57649915763257
    d3 = 88.57649915763257
    h  = 0.1
    alpha = (n1*h)/d1
    beta  = (n2*h)/d2
    gamma = (n3*h)/d3
    return alpha, beta,gamma
alpha,beta,gamma = change_lay3(2,0,0)
print(f'angle at which w1 rotated : Alpha = {alpha}\n'
      f'angle at which w2 rotated : Beta  = {beta}\n'
      f'angle at which w3 rotated : Gamma = {gamma}')

angle at which w1 rotated : Alpha = 0.0008
angle at which w2 rotated : Beta  = 0.0
angle at which w3 rotated : Gamma = 0.0


In [ ]:
# Define the points of the triangle in 3D, only using x and z coordinates
A = np.array([-125, 0, 0]) 
B = np.array([125, 0, 45])
C = np.array([125, 0, -45])

#  Calculate vectors representing the sides in 2D
AB = B-A
BC = C-B
CA = A-C

# Function to calculate angle between two vectors in 2D
def angle_between_vectors(v1, v2):
    cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip to avoid numerical issues
    return angle_rad

#  Calculate angles at each vertex in 2D
angle_A = angle_between_vectors(-CA, AB)

# Print angles
print(f"Angle at A: {angle_A:.10f} ")

Angle at A: 0.3561858765 


In [10]:
def trans_w1(n1):
    d1 = 250
    h  = 0.1
    alpha = (n1*h)/d1
    t = np.array([[-125],[0],[0],[1]])
    T = trans_matrix(-t) @ rot_matrix(-alpha,'z') @ trans_matrix(t)
    return T

def trans_w2(n2):
    d2 = 88.57649915763257
    h  = 0.1
    gamma  = (n2*h)/d2
    t = np.array([[125],[0],[0],[1]])
    rot_a = angle_A / 2
    T = trans_matrix(-t) @ rot_matrix(rot_a,'y') @ rot_matrix(-gamma,'x') @ rot_matrix(-rot_a,'y') @trans_matrix(t)
    return T

def trans_w3(n3):
    d3 = 88.57649915763257
    h  = 0.1
    gamma  = (n3*h)/d3
    t = np.array([[125],[0],[0],[1]])
    rot_a = angle_A / 2
    T = trans_matrix(-t) @ rot_matrix(-rot_a,'y') @ rot_matrix(gamma,'x') @ rot_matrix(rot_a,'y') @trans_matrix(t)
    return T

In [ ]:
m_c = np.array([[0],[205],[0],[1]])
m_b = np.array([[0],[0],[0],[1]])
# print(f'The position of magnet center is\n'
#       f'{m_c[0:3]}')

w1,w2,w3 = [0,0,0]

m_c = trans_w2(w2) @ trans_w3(w3) @ trans_w1(w1) @ m_c
m_b = trans_w3(w3) @ trans_w2(w2) @ trans_w1(w1) @ m_b
delta_y = m_b[1]
pitch =  m_c[2]/205 # around x 
roll =  m_c[0]/205 # around z

# print(f'The position of magnet center is\n'f'{m_c[0:3]}')
print(f'delta x : {m_c[0,0]:.10f}')
print(f'delta y : {delta_y[0]:.10f}')    
print(f'delta z : {m_c[2,0]:.10f}')
print(f'Pitch : {pitch[0]:.10f}')
print(f'Roll : {roll[0]:.10f}')





delta x : 0.2460899409
delta y : 0.1499999640
delta z : 0.0000000000
Pitch : 0.0000000000
Roll : 0.0012004387


In [22]:
import sympy as sp
import numpy as np
def trans_matrix(t):
    M = sp.Matrix([[1,0,0,t[0,0]],
                   [0,1,0,t[1,0]],
                   [0,0,1,t[2,0]],
                   [0,0,0,1     ]])
    return M

def rot_matrix(angle,axis='y'):
    if axis == 'x':
        R = sp.Matrix([
        [1, 0, 0, 0],
        [0,1,-angle, 0],
        [0,angle, 1, 0],
        [0, 0, 0, 1]])
    elif axis == 'y':
        R = sp.Matrix([
        [1, 0, angle, 0],
        [0, 1, 0, 0],
        [-angle, 0, 1, 0],
        [0, 0, 0, 1]])
    elif axis == 'z':
        R = sp.Matrix([
        [1,-angle,0, 0],
        [angle, 1,0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])
    return R
def rot_matrix(angle,axis='y'):
    if axis == 'x':
        R = sp.Matrix([
        [1, 0, 0, 0],
        [0,1,-angle, 0],
        [0,angle, 1, 0],
        [0, 0, 0, 1]])
    elif axis == 'y':
        R = sp.Matrix([
        [1, 0, angle, 0],
        [0, 1, 0, 0],
        [-angle, 0, 1, 0],
        [0, 0, 0, 1]])
    elif axis == 'z':
        R = sp.Matrix([
        [1,-angle,0, 0],
        [angle, 1,0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])
    return R

def trans_w1(n1):
    d1 = 250
    h  = 0.1
    alpha = (n1*h)/d1
    t = np.array([[125],[0],[0],[1]])
    T = trans_matrix(-t) @ rot_matrix(alpha,'z') @ trans_matrix(t)
    return T

def trans_w2(n2):
    d2 = 88.57649915763257
    h  = 0.1
    gamma  = (n2*h)/d2
    t = np.array([[-125],[0],[0],[1]])
    angle_A = 0.3561858765 #angle between wedge12 and 13
    rot_a = angle_A / 2
    T = trans_matrix(-t) @ rot_matrix(-rot_a,'y') @ rot_matrix(-gamma,'x') @ rot_matrix(rot_a,'y') @trans_matrix(t)
    return T

def trans_w3(n3):
    d3 = 88.57649915763257
    h  = 0.1
    gamma  = (n3*h)/d3
    t = np.array([[-125],[0],[0],[1]])
    angle_A = 0.3561858765 #angle between wedge12 and 13
    rot_a = angle_A / 2
    T = trans_matrix(-t) @ rot_matrix(rot_a,'y') @ rot_matrix(gamma,'x') @ rot_matrix(-rot_a,'y') @trans_matrix(t)
    return T
m_c = np.array([[0],[205],[0],[1]])
n1,n2,n3 = sp.symbols('n_1,n_2,n_3', real=True) 

final_position = trans_w1(n1)@trans_w2(n2)@trans_w3(n3)@m_c

# Step 1: Simplify the expression
simplified_position = sp.simplify(final_position)
# Threshold for small terms
threshold = 1e-4

# Define a function to approximate symbolic terms
def filter_small_terms(expr):
    # For Add (sum of terms), filter each term
    if expr.is_Add:
        filtered_terms = [t for t in expr.as_ordered_terms() 
                          if not (t.is_number and abs(t) < threshold) and
                             not (t.as_coeff_Mul()[0].is_number and abs(t.as_coeff_Mul()[0]) < threshold)]
        return sp.Add(*filtered_terms)
    # For Mul (product of terms), check numerical coefficient
    elif expr.is_Mul:
        coeff, _ = expr.as_coeff_Mul()
        return 0 if coeff.is_number and abs(coeff) < threshold else expr
    # For numbers, apply the threshold directly
    elif expr.is_number:
        return 0 if abs(expr) < threshold else expr
    return expr  # Return symbolic terms unchanged

# Apply the filter to each element in the matrix
approx_position = simplified_position.applyfunc(filter_small_terms)
approx_position


Matrix([
[                                -0.082*n_1 + 0.0412175381601814*n_2 + 0.0425248387194341*n_3 - 8.055020425295],
[0.046777991829882*n_1 + 0.000252999173645838*n_2*n_3 + 0.025929779706972*n_2 + 0.0251326452196228*n_3 + 205.0],
[                                                               -0.231438363391601*n_2 + 0.238778915869979*n_3],
[                                                                                                            1]])

In [50]:
def trans_matrix(t):
    M = sp.Matrix([[1,0,0,t[0,0]],
                   [0,1,0,t[1,0]],
                   [0,0,1,t[2,0]],
                   [0,0,0,1     ]])
    return M

def rot_matrix(angle,axis='y'):
    if axis == 'x':
        R = np.array([
        [1, 0, 0, 0],
        [0,np.cos(angle),-np.sin(angle), 0],
        [0,np.sin(angle), np.cos(angle), 0],
        [0, 0, 0, 1]])
    elif axis == 'y':
        R = np.array([
        [np.cos(angle), 0, np.sin(angle), 0],
        [0, 1, 0, 0],
        [-np.sin(angle), 0, np.cos(angle), 0],
        [0, 0, 0, 1]])
    elif axis == 'z':
        R = np.array([
        [np.cos(angle),-np.sin(angle),0, 0],
        [np.sin(angle), np.cos(angle),0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])
    else:
        R = np.eye(4,4)
        print('invalid axis given')
    return R

def trans_w1(n1):
    d1 = 250
    h  = 0.1
    alpha = (n1*h)/d1
    t = np.array([[125],[0],[0],[1]])
    T = trans_matrix(-t) @ rot_matrix(alpha,'z') @ trans_matrix(t)
    return T

def trans_w2(n2):
    d2 = 88.57649915763257
    h  = 0.1
    gamma  = (n2*h)/d2
    t = np.array([[-125],[0],[0],[1]])
    angle_A = 0.3561858765 #angle between wedge12 and 13
    rot_a = angle_A / 2
    T = trans_matrix(-t) @ rot_matrix(-rot_a,'y') @ rot_matrix(-gamma,'x') @ rot_matrix(rot_a,'y') @trans_matrix(t)
    return T

def trans_w3(n3):
    d3 = 88.57649915763257
    h  = 0.1
    gamma  = (n3*h)/d3
    t = np.array([[-125],[0],[0],[1]])
    angle_A = 0.3561858765 #angle between wedge12 and 13
    rot_a = angle_A / 2
    T = trans_matrix(-t) @ rot_matrix(rot_a,'y') @ rot_matrix(gamma,'x') @ rot_matrix(-rot_a,'y') @trans_matrix(t)
    return T

m_c = np.array([[0],[205],[0],[1]])
m_b = np.array([[0],[0],[0],[1]])
w1 = np.array([[125], [0], [0],[1]]) 
w2 = np.array([[-125], [0], [45],[1]])
w3 = np.array([[-125], [0], [-45],[1]])
# trans_w3(-10) @ trans_w2(10) @ trans_w1(10) @ m_b 
print(trans_w1(10) @ trans_w3(10) @ trans_w1(-10) @ m_b)
print(trans_w3(10) @ m_b)

Matrix([[-0.00175008229172338], [0.250023552964595], [-0.00416653747812981], [1.00000000000000]])
Matrix([[0.000249997344724306], [0.249994689360370], [0.00138887413714608], [1.00000000000000]])
